In [10]:
#pip install git+https://github.com/promptslab/Promptify.git

In [11]:
#pip install pandas

In [1]:
#pip install openpyxl

In [1]:
import pandas as pd
import numpy as np
from promptify import OpenAI
from promptify import Prompter

C:\Users\sanka\anaconda3\envs\myenv1\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
#cybersecurity articles in excelsheet to dataframe
fileName = 'th_db_sub.xlsx'
df1 = pd.read_excel(fileName,sheet_name="Sheet1")
df1=df1[1:2]

In [15]:
df1

,Sentence
1,The nation-state adversary group known as FANC...


In [18]:
# Create an empty DataFrame to store the results of NLP model
all_results_df = pd.DataFrame()

for index, row in df1.iterrows():
    sentence = df1["Sentence"][index]


    model = OpenAI(api_key="xxx")
    nlp_prompter = Prompter(model)
    result = nlp_prompter.fit('ner.jinja',
                          domain      = 'cybersecurity',
                          text_input  = sentence, 
                          labels      = ['Threatactor', 'Aliases', 'Targets', 'Tools/Technique',
       'BelievedOrigin'])

    #print(eval(result['text']))

    parsed_data = []
    for item in result:
        completion = item['parsed']['data']['completion']
        parsed_data.extend(completion)
    #print(parsed_data ) 
    df=pd.DataFrame.from_records(parsed_data)

    df=df[['T','E']]
    # change the values in the 'column_name' column to uppercase
    df['E'] = df['E'].str.upper()
    df=df.dropna()
    # rename columns
    df = df.rename(columns={'T': 'Label', 'E': 'Node Name'})

    # Get the 'Threatactor' row
    threatactor_row = df[df['Label'] == 'Threatactor']

    # Set the 'Topic' column based on the 'Node Name' column, using the 'Label' value from the 'Threatactor' row
    df['Topic'] = np.where(df['Label'] == 'Threatactor', '', threatactor_row['Node Name'].iloc[0])

    # Concatenate the result_df to the all_results_df DataFrame
    all_results_df = pd.concat([all_results_df, df], ignore_index=True)

print(all_results_df)
  

624 3376
              Label                                          Node Name   
0       Threatactor                                         FANCY BEAR  \
1           Aliases                                              APT28   
2           Aliases                                             SOFACY   
3           Targets                                          AEROSPACE   
4           Targets                                            DEFENSE   
5           Targets                                             ENERGY   
6           Targets                                         GOVERNMENT   
7           Targets                                              MEDIA   
8           Targets                                         DISSIDENTS   
9   Tools/Technique                                  PHISHING MESSAGES   
10  Tools/Technique                              CREDENTIAL HARVESTING   
11  Tools/Technique                                             XAGENT   
12  Tools/Technique          

In [19]:
df=all_results_df


In [20]:
# Look at unique labels
df.Label.unique()


array(['Threatactor', 'Aliases', 'Targets', 'Tools/Technique',
       'BelievedOrigin'], dtype=object)

In [21]:
#drop the duplicates
df=df.drop_duplicates()

In [22]:
# Build the vertices (nodes) - 
VERTICES = []
EDGES = []


In [23]:
# vertices for Thraetactor label
threat_actor = df[df.Label=='Threatactor']
threat_actor = threat_actor.drop(columns=['Topic'])
threat_actor=threat_actor.drop_duplicates()
for index,threat in threat_actor.iterrows():
    VERTICES.append(f"g.addV('{threat.Label}').property('name','{threat['Node Name']}').property('type','Threatactor')")

In [24]:
# vertices for Aliases label
threat_aliases = df[df.Label=='Aliases']
threat_aliases = threat_aliases.drop(columns=['Topic'])
threat_aliases=threat_aliases.drop_duplicates()
for index,threatas in threat_aliases.iterrows():
    VERTICES.append(f"g.addV('{threatas.Label}').property('name','{threatas['Node Name']}').property('type','Threatactor')")

In [25]:
# vertices for Thraetactor Targets
threat_targets = df[df.Label=='Targets']
threat_targets = threat_targets.drop(columns=['Topic'])
threat_targets = threat_targets.drop_duplicates()
for index,target in threat_targets.iterrows():
    VERTICES.append(f"g.addV('{target.Label}').property('name','{target['Node Name']}').property('type','Threatactor')")

In [26]:

# vertices for Thraetactor Tools/Technique
threat_tools = df[df.Label=='Tools/Technique']
threat_tools = threat_tools.drop(columns=['Topic'])
threat_tools = threat_tools.drop_duplicates()
for index,tool in threat_tools.iterrows():
    VERTICES.append(f"g.addV('{tool.Label}').property('name','{tool['Node Name']}').property('type','Threatactor')")

In [27]:
# vertices for Thraetactor BelievedOrigin
threat_origin = df[df.Label=='BelievedOrigin']
threat_origin = threat_origin.drop(columns=['Topic'])
threat_origin = threat_origin.drop_duplicates()
for index,origin in threat_origin.iterrows():
    VERTICES.append(f"g.addV('{origin.Label}').property('name','{origin['Node Name']}').property('type','Threatactor')")

In [28]:
# Add Targets to Threatactor
target_edge = df[df.Label=='Targets']
for index, tar in target_edge.iterrows():
    EDGES.append(f"g.V().hasLabel('Threatactor').has('name','{tar['Topic']}').addE('target_to').to(g.V().hasLabel('Targets').has('name','{tar['Node Name']}'))")


In [29]:
# Add Tools/Technique to Threatactor
tool_edge = df[df.Label=='Tools/Technique']
for index, tool in tool_edge.iterrows():
    EDGES.append(f"g.V().hasLabel('Threatactor').has('name','{tool['Topic']}').addE('uses').to(g.V().hasLabel('Tools/Technique').has('name','{tool['Node Name']}'))")


In [30]:
# Add BelievedOrigin to Threatactor
origin_edge = df[df.Label=='BelievedOrigin']
for index, org in origin_edge.iterrows():
    EDGES.append(f"g.V().hasLabel('Threatactor').has('name','{org['Topic']}').addE('belongs_to').to(g.V().hasLabel('BelievedOrigin').has('name','{org['Node Name']}'))")


In [31]:
# Add Aliases(out) to Threatactor(in)
aliases_edge = df[df.Label=='Aliases']
for index, sub in aliases_edge.iterrows():
    EDGES.append(f"g.V().hasLabel('Aliases').has('name','{sub['Node Name']}').addE('aliases_is').to(g.V().hasLabel('Threatactor').has('name','{sub['Topic']}'))")


In [32]:
VERTICES

["g.addV('Threatactor').property('name','FANCY BEAR').property('type','Threatactor')",
 "g.addV('Aliases').property('name','APT28').property('type','Threatactor')",
 "g.addV('Aliases').property('name','SOFACY').property('type','Threatactor')",
 "g.addV('Targets').property('name','AEROSPACE').property('type','Threatactor')",
 "g.addV('Targets').property('name','DEFENSE').property('type','Threatactor')",
 "g.addV('Targets').property('name','ENERGY').property('type','Threatactor')",
 "g.addV('Targets').property('name','GOVERNMENT').property('type','Threatactor')",
 "g.addV('Targets').property('name','MEDIA').property('type','Threatactor')",
 "g.addV('Targets').property('name','DISSIDENTS').property('type','Threatactor')",
 "g.addV('Tools/Technique').property('name','PHISHING MESSAGES').property('type','Threatactor')",
 "g.addV('Tools/Technique').property('name','CREDENTIAL HARVESTING').property('type','Threatactor')",
 "g.addV('Tools/Technique').property('name','XAGENT').property('type','

In [33]:
EDGES

["g.V().hasLabel('Threatactor').has('name','FANCY BEAR').addE('target_to').to(g.V().hasLabel('Targets').has('name','AEROSPACE'))",
 "g.V().hasLabel('Threatactor').has('name','FANCY BEAR').addE('target_to').to(g.V().hasLabel('Targets').has('name','DEFENSE'))",
 "g.V().hasLabel('Threatactor').has('name','FANCY BEAR').addE('target_to').to(g.V().hasLabel('Targets').has('name','ENERGY'))",
 "g.V().hasLabel('Threatactor').has('name','FANCY BEAR').addE('target_to').to(g.V().hasLabel('Targets').has('name','GOVERNMENT'))",
 "g.V().hasLabel('Threatactor').has('name','FANCY BEAR').addE('target_to').to(g.V().hasLabel('Targets').has('name','MEDIA'))",
 "g.V().hasLabel('Threatactor').has('name','FANCY BEAR').addE('target_to').to(g.V().hasLabel('Targets').has('name','DISSIDENTS'))",
 "g.V().hasLabel('Threatactor').has('name','FANCY BEAR').addE('uses').to(g.V().hasLabel('Tools/Technique').has('name','PHISHING MESSAGES'))",
 "g.V().hasLabel('Threatactor').has('name','FANCY BEAR').addE('uses').to(g.V().

In [34]:
#!pip install gremlinpython

In [15]:
from gremlin_python.driver import client, serializer, protocol
from gremlin_python.driver.protocol import GremlinServerError
import sys
import traceback
import asyncio
import collections.abc as collections


In [16]:
#!az login


In [17]:
#To populate the data into cosmosdb
ENDPOINT = 'wss://cosmosdbinstance1.gremlin.cosmos.azure.com'
PRIMARY_KEY ="xxx"
DATABASE = 'threatactordb'
COLLECTION = 'threatactorgraph'

In [19]:

# This code is based on azure sdk for python

if sys.platform == 'win32':
    asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())

#_gremlin_cleanup_graph = "g.V().drop()"

_gremlin_insert_vertices =VERTICES

_gremlin_insert_edges =EDGES

#_gremlin_update_vertices = []

#_gremlin_count_vertices = "g.V().count()"

#_gremlin_traversals = {}

#_gremlin_drop_operations ={}

def print_status_attributes(result):
    # This logs the status attributes returned for successful requests.
    # See list of available response status attributes (headers) that Gremlin API can return:
    #     https://docs.microsoft.com/en-us/azure/cosmos-db/gremlin-headers#headers
    #
    # These responses includes total request units charged and total server latency time.
    # 
    # IMPORTANT: Make sure to consume ALL results returend by cliient tothe final status attributes
    # for a request. Gremlin result are stream as a sequence of partial response messages
    # where the last response contents the complete status attributes set.
    #
    # This can be 
    print("\tResponse status_attributes:\n\t{0}".format(result.status_attributes))

def cleanup_graph(client):
    print("\n> {0}".format(
        _gremlin_cleanup_graph))
    callback = client.submitAsync(_gremlin_cleanup_graph)
    if callback.result() is not None:
        callback.result().all().result() 
    print("\n")
    print_status_attributes(callback.result())
    print("\n")


def insert_vertices(client):
    for query in _gremlin_insert_vertices:
        print("\n> {0}\n".format(query))
        callback = client.submitAsync(query)
        if callback.result() is not None:
            print("\tInserted this vertex:\n\t{0}".format(
                callback.result().all().result()))
        else:
            print("Something went wrong with this query: {0}".format(query))
        print("\n")
        print_status_attributes(callback.result())
        print("\n")

    print("\n")


def insert_edges(client):
    for query in _gremlin_insert_edges:
        print("\n> {0}\n".format(query))
        callback = client.submitAsync(query)
        if callback.result() is not None:
            print("\tInserted this edge:\n\t{0}\n".format(
                callback.result().all().result()))
        else:
            print("Something went wrong with this query:\n\t{0}".format(query))
        print_status_attributes(callback.result())
        print("\n")

    print("\n")


def update_vertices(client):
    for query in _gremlin_update_vertices:
        print("\n> {0}\n".format(query))
        callback = client.submitAsync(query)
        if callback.result() is not None:
            print("\tUpdated this vertex:\n\t{0}\n".format(
                callback.result().all().result()))
        else:
            print("Something went wrong with this query:\n\t{0}".format(query))

        print_status_attributes(callback.result())
        print("\n")

    print("\n")


def count_vertices(client):
    print("\n> {0}".format(
        _gremlin_count_vertices))
    callback = client.submitAsync(_gremlin_count_vertices)
    if callback.result() is not None:
        print("\tCount of vertices: {0}".format(callback.result().all().result()))
    else:
        print("Something went wrong with this query: {0}".format(
            _gremlin_count_vertices))

    print("\n")
    print_status_attributes(callback.result())
    print("\n")


def execute_traversals(client):
    for key in _gremlin_traversals:
        print("{0}:".format(key))
        print("> {0}\n".format(
            _gremlin_traversals[key]))
        callback = client.submitAsync(_gremlin_traversals[key])
        for result in callback.result():
            print("\t{0}".format(str(result)))
        
        print("\n")
        print_status_attributes(callback.result())
        print("\n")


def execute_drop_operations(client):
    for key in _gremlin_drop_operations:
        print("{0}:".format(key))
        print("\n> {0}".format(
            _gremlin_drop_operations[key]))
        callback = client.submitAsync(_gremlin_drop_operations[key])
        for result in callback.result():
            print(result)
        print_status_attributes(callback.result())
        print("\n")


try:
    client = client.Client('wss://cosmosauto12.gremlin.cosmos.azure.com:443/', 'g',
                           username="/dbs/graphdb/colls/graph1",
                           password="xxx",
                           message_serializer=serializer.GraphSONSerializersV2d0()
                           )

    print("Welcome to Azure Cosmos DB + Gremlin on Python!")

    # Drop the entire Graph
    #input("We're about to drop whatever graph is on the server. Press any key to continue...")
    #cleanup_graph(client)

    # Insert all vertices
    input("Let's insert some vertices into the graph. Press any key to continue...")
    insert_vertices(client)

    #Create edges between vertices
    input("Now, let's add some edges between the vertices. Press any key to continue...")
    insert_edges(client)

    # Update a vertex
    #input("Ah, sorry. I made a mistake. Let's change the age of this vertex. Press any key to continue...")
    #update_vertices(client)

    # Count all vertices
    #input("Okay. Let's count how many vertices we have. Press any key to continue...")
    #count_vertices(client)

    # Execute traversals and get results
    #input("Cool! Let's run some traversals on our graph. Press any key to continue...")
    #execute_traversals(client)

    # Drop a few vertices and edges
    #input("So, life happens and now we will make some changes to the graph. Press any key to continue...")
    #execute_drop_operations(client)

    # Count all vertices again
    #input("How many vertices do we have left? Press any key to continue...")
    #count_vertices(client)

except GremlinServerError as e:
    print('Code: {0}, Attributes: {1}'.format(e.status_code, e.status_attributes))

    # GremlinServerError.status_code returns the Gremlin protocol status code
    # These are broad status codes which can cover various scenaios, so for more specific
    # error handling we recommend using GremlinServerError.status_attributes['x-ms-status-code']
    # 
    # Below shows how to capture the Cosmos DB specific status code and perform specific error handling.
    # See detailed set status codes than can be returned here: https://docs.microsoft.com/en-us/azure/cosmos-db/gremlin-headers#status-codes
    #
    # See also list of available response status attributes that Gremlin API can return:
    #     https://docs.microsoft.com/en-us/azure/cosmos-db/gremlin-headers#headers
    cosmos_status_code = e.status_attributes["x-ms-status-code"]
    if cosmos_status_code == 409:
        print('Conflict error!')
    elif cosmos_status_code == 412:
        print('Precondition error!')
    elif cosmos_status_code == 429:
        print('Throttling error!');
    elif cosmos_status_code == 1009:
        print('Request timeout error!')
    else:
        print("Default error handling")

    traceback.print_exc(file=sys.stdout) 
    sys.exit(1)

print("\nAnd that's all! Sample complete")
input("Press Enter to continue...")


Welcome to Azure Cosmos DB + Gremlin on Python!
We're about to drop whatever graph is on the server. Press any key to continue...

> g.V().drop()


	Response status_attributes:
	{'x-ms-status-code': 200, 'x-ms-activity-id': 'ba6d998d-0403-4d60-8091-265e88f5c55d', 'x-ms-request-charge': 2.79, 'x-ms-total-request-charge': 2.79, 'x-ms-server-time-ms': 42.8645, 'x-ms-total-server-time-ms': 42.8645}


Let's insert some vertices into the graph. Press any key to continue...

> g.addV('Threatactor').property('name','CrowdStrike Intelligence').property('type','Threatactor')

	Inserted this vertex:
	[{'id': '2f926218-893a-4245-8045-c76bce517a29', 'label': 'Threatactor', 'type': 'vertex', 'properties': {'name': [{'id': 'b497ebba-25d2-4299-a4ca-a1c6c96dafca', 'value': 'CrowdStrike Intelligence'}], 'type': [{'id': '2f926218-893a-4245-8045-c76bce517a29|type', 'value': 'Threatactor'}]}}]


	Response status_attributes:
	{'x-ms-status-code': 200, 'x-ms-activity-id': 'ea3ccaff-a9d0-4aa0-a708-c983219040d

	Inserted this edge:
	[{'id': '146c1545-6dba-4b0c-b6e1-b721277941b6', 'label': 'uses', 'type': 'edge', 'inVLabel': 'Tools/Technique', 'outVLabel': 'Threatactor', 'inV': 'b58a8870-bb4e-4024-bbc5-055a82cf1186', 'outV': '6366b3ca-fa0b-459a-a443-5684057f7ecf'}]

	Response status_attributes:
	{'x-ms-status-code': 200, 'x-ms-activity-id': '9f87bffe-c4d2-47c9-bc4a-9a8c80aaf494', 'x-ms-request-charge': 14.620000000000001, 'x-ms-total-request-charge': 14.620000000000001, 'x-ms-server-time-ms': 27.5926, 'x-ms-total-server-time-ms': 27.5926}



> g.V().hasLabel('Threatactor').has('name','Intelligence').addE('uses').to(g.V().hasLabel('Tools/Technique').has('name','Penetration Testing Tools'))

	Inserted this edge:
	[{'id': 'ced7f819-a26f-49bc-9b66-20384f13e650', 'label': 'uses', 'type': 'edge', 'inVLabel': 'Tools/Technique', 'outVLabel': 'Threatactor', 'inV': 'aa57836f-ee49-4e7e-8d2b-74a37d6bc3f9', 'outV': '6366b3ca-fa0b-459a-a443-5684057f7ecf'}]

	Response status_attributes:
	{'x-ms-status-code':

''